In [0]:
#%%
import requests
from typing import Dict, Any, Tuple
import logging
import pandas as pd
import csv
import json
import os
from datetime import datetime
from pyspark.sql.functions import col, to_timestamp, year, month, day, hour, minute
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:

def ler_csv_variaveis_API() -> Tuple[int, int, str]:
    try:
        df = spark.read.csv(caminho_arquivo, header=False)
        if df.count() > 0:
            row = df.collect()[0]
            pagina = int(row[0])
            porPagina = int(row[1])
            estrategia = str(row[2])
            return pagina, porPagina, estrategia
        else:
            print(f"Arquivo vazio: {caminho_arquivo}")
            return None
    except Exception as e:
        print(f"Erro ao ler o arquivo: {caminho_arquivo} - {str(e)}")
        return None


def buscar_conteudo_TabNews (pagina: int, porPagina:int, estrategia:str) -> Dict[str, Any]:
    """
    Busca os conteudos da API do TabNews. 
    Args:
        pagina (int): O n├║mero da p├ígina a ser buscada. Padr├úo ├® 1.
        por_pagina (int): N├║mero de itens por p├ígina. Padr├úo ├® 10.
        estrategia (str): Estrat├®gia de conte├║do ('new', 'old', 'relevant'). Padr├úo ├® 'new'.
    Returns:
        Dict[str, Any]: Resposta JSON da API.
    Raises:
        requests.RequestException: Se a requisi├º├úo ├á API falhar.
    """
    urlbase= f'https://www.tabnews.com.br/api/v1/contents?page={pagina}&per_page={porPagina}&strategy={estrategia}'
    try:
        resposta = requests.get(url=urlbase)
        return resposta.text
    except requests.RequestException as erro:
        logging.error(f'Erro ao buscar dados da API do TabNews: {erro}')


def salvar_json(json_data: str):
    parsed_json = json.loads(json_data)
    try:
        for objeto_json in parsed_json:
            created_at = datetime.fromisoformat(objeto_json['created_at'].replace('Z', '+00:00'))
            caminho_pastas = f"{caminho_raw}/Ano_{created_at.year}/M├¬s_{created_at.month:02d}/Dia_{created_at.day:02d}/Minuto_{created_at.hour:02d}/Segundo_{created_at.minute:02d}"
            nome_arquivo_json = f"{objeto_json['id']}.json"
            caminho_arquivo_json = f"{caminho_pastas}/{nome_arquivo_json}"
            
            # Criar diret├│rios
            dbutils.fs.mkdirs(caminho_pastas)
            
            # Salvar JSON
            dbutils.fs.put(caminho_arquivo_json, json.dumps(objeto_json, ensure_ascii=False, indent=2), overwrite=True)
    except KeyError as e:
        print(f'Tivemos um erro para ler a chave no objeto JSON - {e}')
    except Exception as e:
        print(f'Erro inesperado - {e}')

def adicionar_contador(pagina: int, porPagina: int, estrategia: str):
    pagina += 1
    novos_dados = [(pagina, porPagina, estrategia)]
    
    schema = StructType([
        StructField("pagina", IntegerType(), False),
        StructField("porPagina", IntegerType(), False),
        StructField("estrategia", StringType(), False)
    ])
    
    df = spark.createDataFrame(novos_dados, schema)
    df.write.mode("overwrite").csv(caminho_arquivo, header=False)

In [0]:
def main():
    for i in range(10):
        pagina, porPagina, estrategia = ler_csv_variaveis_API ()
        resultado = buscar_conteudo_TabNews(pagina, porPagina, estrategia)
        salvar_json(str(resultado))
        adicionar_contador(pagina, porPagina, estrategia)
# %%
if __name__ == '__main__':
    main()